In [1]:
# Author: Meysam Aghighi
# Contact: meysam.aghighi@gmail.com
# Date: 2024-11-08
# Description: A script to automate desk booking in flowscape

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time